# CartPole-v1 Demo

## This activity was done using the DDQN method

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. 
- The system is controlled by applying a force of +1 or -1 to the cart. 
- The pendulum starts upright, and the goal is to prevent it from falling over. 
- A reward of +1 is provided for every timestep that the pole remains upright. 
- The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

## Load Packages

In [6]:
import gym
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from collections import deque
from IPython.display import HTML

## DDQN Solver Class

In [1]:
import random
from collections import deque

import numpy as np
import tensorflow as tf


class DDQN_Solver():
    def __init__(self, gamma, memory_size, min_memory_size, learning_rate_adam, HL_1_size, HL_2_size, batch_size,
                 epsilon_all):
        # Load important parameters
        self.gamma = gamma  # discount rate
        self.memory_size = memory_size  # size of the memory buffer
        self.HL_1_size = HL_1_size  # number of nodes in the first hidden layer
        self.HL_2_size = HL_2_size  # number of nodes in the second hidden layer
        self.learning_rate_adam = learning_rate_adam  # learning rate for Adam optimizer
        self.batch_size = batch_size  # batch size for training
        self.min_memory_size = max(self.batch_size, min_memory_size)  # minimal memory size before we start training
        self.epsilon_initial = epsilon_all['initial']  # epsilon-greedy policy - initial value
        self.epsilon_decay = epsilon_all['decay']  # decay after each time step
        self.epsilon_min = epsilon_all['min']  # minimal value of epsilon

        # Initialize attributes
        self.replay_buffer = deque()
        self.global_step = 0  # counts the number of times we have trained our model = sum_{episode} timesteps_episode
        self.most_recent_score = tf.Variable(0, dtype=tf.int32)  # most recent score - visualized in tensorboard
        tf.summary.scalar('most_recent_score', self.most_recent_score)
        self.epsilon = self.epsilon_initial  # we initialize our epsilon
        self.epsilon_tensor = tf.Variable(self.epsilon, dtype=tf.float32)  # for tensorboard
        tf.summary.scalar('epsilon', self.epsilon_tensor)

        # Build online and target networks
        self.__build_Q_net()

        # Merge summaries
        self.overall_summary = tf.summary.merge_all()

        # Initialize variables and summary writer
        self.__init_session()
        self.summary_writer = tf.summary.FileWriter('/ddqn_summaries', self.session.graph)

        # Synchronize Online and Target Network
        self.update_target_network()

    def __build_Q_net(self):
        # Placeholders
        self.input_state = tf.placeholder(tf.float32, [None, 4], 'Input_state')
        self.input_action = tf.placeholder(tf.float32, [None, 2], 'Input_action')
        self.target = tf.placeholder(tf.float32, [None], 'Target')

        # Online Network Variables
        self.W1_on = tf.Variable(tf.truncated_normal([4, self.HL_1_size]))
        self.b1_on = tf.Variable(tf.constant(0.1, shape=[self.HL_1_size]))
        self.HL_1_on = tf.nn.relu(tf.matmul(self.input_state, self.W1_on) + self.b1_on, )
        self.W2_on = tf.Variable(tf.truncated_normal([self.HL_1_size, self.HL_2_size]))
        self.b2_on = tf.Variable(tf.constant(0.1, shape=[self.HL_2_size]))
        self.HL_2_on = tf.nn.relu(tf.matmul(self.HL_1_on, self.W2_on) + self.b2_on)
        self.W3_on = tf.Variable(tf.truncated_normal([self.HL_1_size, 2]))
        self.b3_on = tf.Variable(tf.constant(0.1, shape=[2]))
        self.Q_ohr_on = tf.matmul(self.HL_2_on, self.W3_on) + self.b3_on

        # Target Network Variables
        self.W1_tn = tf.Variable(tf.truncated_normal([4, self.HL_1_size]))
        self.b1_tn = tf.Variable(tf.constant(0.1, shape=[self.HL_1_size]))
        self.HL_1_tn = tf.nn.relu(tf.matmul(self.input_state, self.W1_tn) + self.b1_tn, )
        self.W2_tn = tf.Variable(tf.truncated_normal([self.HL_1_size, self.HL_2_size]))
        self.b2_tn = tf.Variable(tf.constant(0.1, shape=[self.HL_2_size]))
        self.HL_2_tn = tf.nn.relu(tf.matmul(self.HL_1_tn, self.W2_tn) + self.b2_tn)
        self.W3_tn = tf.Variable(tf.truncated_normal([self.HL_1_size, 2]))
        self.b3_tn = tf.Variable(tf.constant(0.1, shape=[2]))
        self.Q_ohr_tn = tf.matmul(self.HL_2_tn, self.W3_tn) + self.b3_tn

        # Q function and loss
        self.Q_on = tf.reduce_sum(tf.multiply(self.Q_ohr_on, self.input_action), reduction_indices=1)

        # Loss
        self.loss = tf.reduce_mean(tf.square(self.target - self.Q_on), name='loss')
        tf.summary.scalar("loss", self.loss)

        # Train operations
        self.train_op = tf.train.AdamOptimizer(self.learning_rate_adam).minimize(self.loss)

    def __init_session(self):
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())

    def train(self):
        """ Samples a minibatch from the memory and based on it trains the network
        """
        self.global_step += 1

        # Just make sure that it breaks at the beginning when memory is not big enough < min_memory_size
        if len(self.replay_buffer) < self.min_memory_size:
            #print('The memory is too small to train')
            return

        # Sample from memory

        mini_batch = random.sample(self.replay_buffer, self.batch_size)  # sampling without replacement
        batch_s_old = [element[0] for element in mini_batch]
        batch_a = [element[1] for element in mini_batch]
        batch_r = [element[2] for element in mini_batch]
        batch_s_new = [element[3] for element in mini_batch]
        batch_d = [element[4] for element in mini_batch]

        # Generating targets
        Q_new_on = self.Q_ohr_on.eval(feed_dict={self.input_state: batch_s_new})  # forward pass - ONLINE NETWORK
        Q_new_tn = self.Q_ohr_tn.eval(feed_dict={self.input_state: batch_s_new})  # forward pass - TARGET NETWORK
        argmax = np.argmax(Q_new_on, axis=1)
        Q_target = np.reshape(np.array([Q_new_tn[i][argmax[i]] for i in range(self.batch_size)]),
                              newshape=self.batch_size)

        # Generate targets
        batch_target = []
        for i in range(self.batch_size):
            if batch_d[i]:
                # The new state is the end game - its target Q value is definitely 0
                batch_target.append(batch_r[i])
            else:
                batch_target.append(batch_r[i] + self.gamma * Q_target[i])

        # Train and write summary
        _, summary_str = self.session.run([self.train_op, self.overall_summary], feed_dict={
            self.target: batch_target,
            self.input_state: batch_s_old,
            self.input_action: batch_a,
        })
        self.summary_writer.add_summary(summary_str, self.global_step)

        # Decay epsilon
        self.__decay_epsilon()

    def update_target_network(self):
        # We simply copy online network values into the target network
        ops_list = []
        ops_list.append(self.W1_tn.assign(self.W1_on))
        ops_list.append(self.b1_tn.assign(self.b1_on))
        ops_list.append(self.W2_tn.assign(self.W2_on))
        ops_list.append(self.b2_tn.assign(self.b2_on))
        ops_list.append(self.W3_tn.assign(self.W3_on))
        ops_list.append(self.b3_tn.assign(self.b3_on))

        self.session.run(ops_list)


    def __decay_epsilon(self, printme=False):
        """ Decays epsilon based on epsilon_decay
        :param printme: print current value of epsilon
        :type printme: bool
        """
        if self.epsilon > self.epsilon_min:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        if printme:
            print('The current value of epsilon is ' + str(self.epsilon))

    def memorize(self, old_state, action, reward, new_state, done):
        """ Inserts the most recent SARS and done into the memory - a is saved in the one hot representation """
        # Convert action to one hot representation
        a_ohr = np.zeros(2)
        a_ohr[action] = 1

        # Make sure they have the right dimensions
        old_state.shape = (4,)
        a_ohr.shape = (2,)
        new_state.shape = (4,)

        # Add into replay_buffer and if necessary pop oldest memory
        memory_element = tuple((old_state, a_ohr, reward, new_state, done))
        self.replay_buffer.append(memory_element)
        if len(self.replay_buffer) > self.memory_size:
            self.replay_buffer.popleft()

    def choose_action(self, old_state, policy_from_online):
        """ Epsilon greedy policy """
        
        # just a forward pass and max
        if np.random.rand() < self.epsilon:
            # Explore
            return np.random.choice([0, 1], 1)[0]
        else:
            # Exploit
            old_state.shape = (1, 4)  # make sure it matches the placeholder shape (None, 4)
            if policy_from_online:
                return np.argmax(self.Q_ohr_on.eval(feed_dict={self.input_state: old_state}))
            else:
                return np.argmax(self.Q_ohr_tn.eval(feed_dict={self.input_state: old_state}))

    def feed_most_recent_score(self, score):
        """ Feeds the most recent score into solver class so that it can be visualizes in tensorboard together with epsilon"""
        
        option1 = self.most_recent_score.assign(score)
        option2 = self.epsilon_tensor.assign(self.epsilon)
        self.session.run([option1, option2])

C:\Users\Jacqui.Muller\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Assign Environment Parameters

In [2]:
number_of_episodes = 2500
render_bool = False
penalize_bad_states = (True, -100)

policy_from_online = True 
param_dict = {'gamma': 1,
              'batch_size': 64,
              'HL_1_size': 24,
              'HL_2_size': 24,
              'memory_size': 2000,
              'min_memory_size': 100,
              'learning_rate_adam': 0.001,
              'epsilon_all': {'initial': 1, 'decay': 999 / 1000, 'min': 0.01}}

## Set Algorithm Termination Criteria

In [3]:
number_of_consecutive_episodes = 5
threshold_average = 490

## Initialise Class

In [4]:
my_solver = DDQN_Solver(**param_dict)
env = gym.make('CartPole-v1')
solved = False
results = []

NameError: name 'gym' is not defined

## Evaluate Algorithm (Main Method)

In [ ]:
number_of_steps = []
episodes = []

for episode in range(number_of_episodes):
    old_state = env.reset()
    done = False
    total = 0
    while not done:
        total += 1

        if render_bool:
            env.render()

        # Choose action
        action = my_solver.choose_action(old_state, policy_from_online)

        # Take action
        new_state, reward, done, _ = env.step(action)

        # Penalize
        if penalize_bad_states[0] and done and total < 500:
            reward = penalize_bad_states[1]

        # Memorize
        my_solver.memorize(old_state, action, reward, new_state, done and total < 500)

        # Train
        if not solved:
            my_solver.train()

        # Move forward
        old_state = new_state

    # Append results and check if solved
    results.append(total)

    if np.mean(results[-min(number_of_consecutive_episodes, episode):]) > threshold_average:
        solved = True
        print('Stable solution found - no more training required!')
        break
    else:
        solved = False
        
    episodes.append(episode)
    number_of_steps.append(total)

    print('The episode %s lasted for %s steps' % (episode, total))
    my_solver.feed_most_recent_score(total)
    my_solver.update_target_network()

## Add Statistics into a Data Frame

In [ ]:
stats_df = pd.DataFrame()

stats_df['episode'] = episodes
stats_df['steps'] = number_of_steps

## Plot Data Frame

In [ ]:
stats_df.plot()
plt.show

## Visualisation

In [7]:
HTML('<img src="cart pole.gif">')